In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [3]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 3, "lift_capacity": 2})
manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60)
print(result)

{'total_rides': 13, 'average_wait': 2.076923076923077, 'agents': [Agent(1) wakeful-prior cfbc28a2-face-4c75-ac43-b35b04c327cf, Agent(2) dynamic-listen dda8b14b-cc1d-4435-88ef-956781310584, Agent(3) fluttering-region eab7798f-ba7f-4dbb-89de-da2dc80f4385]}


In [4]:
lift = manager.lift
lift.total_chairs()

50

## Retrieve Data from Simulation

In [5]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

## Sample Agent and Get Exp Traceback

In [6]:
# sample agent and subset exp/log to e/l for agent subset of those
m = log['event'] == 'ride_complete'
event_ride_comp = log[m].sample().iloc[0].to_dict()
agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
l = log[log['agent_uuid'] == agent.agent_uuid]
idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
e = exp.loc[idx].to_dict()

In [8]:
x = agent.traceback_experience(e['exp_id'])
#print(x)

# Agent State Identification (ride loop exp)

In [9]:
# get ride loop logs and agent event logs
log = manager.retrieve_data()
exp = log['exp_rideloop'] # dataframe of all agent rideloop experiences 
log = log['agent_log'] # combined agent logs (i.e. activity logs for all agents)

# sample a random timestamp within the agent log timeframe
INDEX_TIME = pd.Series(pd.date_range(log['time'].min(), log['time'].max(), freq='1s')).sample().iloc[0]

# sample a random agent from ride loop log
row = exp.sample().iloc[0]
aid = row['agent_uuid']
agent = manager.lookup_agent(aid)

# find the nearest time index in the agent log
l = log.zero_liftsim.subset_agent_uuid(agent.agent_uuid)
e = exp.zero_liftsim.subset_agent_uuid(agent.agent_uuid)
idx = l.zero_liftsim.get_nearest_time_index(INDEX_TIME)

for the ride loop agent experience, the total time is broken down into riding the lift, traversing down the mountain and waiting in the queue, i.e. "time_spent_total": ride + traverse + queue below:

In [10]:
s = inspect.getsource(AgentRideLoopExperience)
lines = s.split('\n')
idx = [i for i,x in enumerate(lines) if x.strip() == '"""'][-1]
print('\n'.join(lines[idx+1:]))

    def add_entry(self, agent, return_event_uuid, dt: datetime, ride: float, traverse: float, 
                  queue: float) -> None:
        self.log[dt] = {
            "exp_id": str(uuid()), 
            "return_event_uuid": return_event_uuid,
            "agent_uuid": agent.agent_uuid,
            "agent_uuid_codename": agent.agent_uuid_codename,
            "time_spent_ride_lift": ride,
            "time_spent_traverse_down_mountain": traverse,
            "time_spent_in_queue": queue,
            "time_spent_total": ride + traverse + queue,
        }

